In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint
from scipy.stats import linregress
from datetime import datetime
from yelpapi import YelpAPI
import json

# Impor the API key
from config import api_key

In [ ]:
# Old version, hit many issues with offset
# yelp_api = YelpAPI(api_key)
# search_term = 'restaurants'
# location = 'New York City, NY'
# limit = 50  # maximum number of results to return
# daily_results_limit = 12500 # update this by 12500 every day
# offset = 0 # update this by 12500 every day

# # response = yelp_api.search_query(term=search_term, location=location, limit=limit)
# # pprint(response)
# results = []
# api_call_limit = 10  # total API call limit for the day
# calls_made = 0  # counter for API calls made

# while offset < daily_results_limit and calls_made < api_call_limit:
#     response = yelp_api.search_query(term=search_term, location=location, limit=limit, offset=offset)
#     results.extend(response['businesses'])  # Extend the list with new businesses
#     offset += limit  # Increment the offset by the limit
#     calls_made += 1  # Increment the API call counter
    
#     # Print progress
#     print(f'Retrieved {len(results)} results with {calls_made} API calls')

#     # Avoid hitting rate limits by adding a delay
#     time.sleep(1)  # Delay for 1 second

In [2]:
# NYC boundaries (lat, lon)
northwest = (40.893079105643025, -73.9128920326231)
northeast = (40.879497836080574, -73.82796941055598)
southwest = (40.62030456986446, -74.03292689266031)
southeast = (40.65979392491511, -73.74149354297585)

# Step size for approx 2 km
lat_step = 0.018
lon_step = 0.024

# Generate grid points
lat_points = np.arange(southwest[0], northwest[0], lat_step)
lon_points = np.arange(southwest[1], southeast[1], lon_step)

# Create list of points with increased radius
grid_points_with_radius = [{"lat": lat, "lon": lon, "radius": 1000} for lat in lat_points for lon in lon_points]

# Filter points to ensure they fall within the specified quadrilateral
def is_within_boundaries(lat, lon):
    return (southwest[0] <= lat <= northwest[0]) and (southwest[1] <= lon <= southeast[1])
filtered_grid_points = [point for point in grid_points_with_radius if is_within_boundaries(point["lat"], point["lon"])]

# Number of points generated
num_points = len(filtered_grid_points)

# Split points into four parts for four days
num_days = 4
points_per_day = num_points // num_days

# Ensure all points are covered, even if not divisible by 4
split_grid_points = [filtered_grid_points[i:i + points_per_day] for i in range(0, num_points, points_per_day)]

# Handle any remaining points if num_points is not perfectly divisible by num_days
if len(split_grid_points) > num_days:
    split_grid_points[num_days - 1].extend(split_grid_points.pop())

print(f"Generated {num_points} grid points.")
for day in range(num_days):
    starting_point = split_grid_points[day][0]
    print(f"Day {day + 1}, {len(split_grid_points[day])} grid points, starting point: Latitude {starting_point['lat']}, Longitude {starting_point['lon']}")

# split_grid_points

Generated 208 grid points.
Day 1, 52 grid points, starting point: Latitude 40.62030456986446, Longitude -74.03292689266031
Day 2, 52 grid points, starting point: Latitude 40.69230456986446, Longitude -74.03292689266031
Day 3, 52 grid points, starting point: Latitude 40.764304569864464, Longitude -74.03292689266031
Day 4, 52 grid points, starting point: Latitude 40.83630456986447, Longitude -74.03292689266031


In [ ]:
#Test API Calls
# headers = {
#     "accept": "application/json",
#     "Authorization": f"Bearer {api_key}"
# }
# url_first_call = f"https://api.yelp.com/v3/businesses/search?latitude=40.62030456986446&longitude=-74.03292689266031&term=restaurants&radius=1000&sort_by=distance&limit=50"
# first_response = requests.get(url_first_call, headers=headers).json()
# pprint(first_response)

In [3]:
day = 3 # change this +1 next day
calls_made = 0 # keep track of calls made for display and to esnure we are not going past limit
search_term = 'restaurants' # variable used for api url
limit = 50 # variable used for api url, makes sure we get 50 restaurants per call
offset_limit = 200 # ensure we check the first 200 restaurants within radius of each point
sort_by = 'distance' # variable used for api url
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {api_key}"
}
results = []
# loop over every coordiante point built above for the selected day
print('-----------Starting with API calls for the Day-----------')  
for coord in split_grid_points[day]:
    offset = 0 # used to make sure we look at different restaurants on next api call at same coord
    # call api 4 times per coord
    while offset < offset_limit:
        url = f"https://api.yelp.com/v3/businesses/search?latitude={coord['lat']}&longitude={coord['lon']}&term={search_term}&radius={coord['radius']}&sort_by={sort_by}&limit={limit}&offset={offset}"
        response = requests.get(url, headers=headers).json()
        results.extend(response['businesses'])
        offset += 50 # increase offset for next call
        calls_made += 1 # keep track of calls made for display
        time.sleep(1) # sleep to not get timed out for too many fast calls
    print(f'Retrieved {len(results)} results with {calls_made} API calls') 
print('-----------Finished with API calls for the Day-----------')  

-----------Starting with API calls for the Day-----------
Retrieved 6 results with 4 API calls
Retrieved 67 results with 8 API calls
Retrieved 128 results with 12 API calls
Retrieved 148 results with 16 API calls
Retrieved 348 results with 20 API calls
Retrieved 548 results with 24 API calls
Retrieved 686 results with 28 API calls
Retrieved 886 results with 32 API calls
Retrieved 1042 results with 36 API calls
Retrieved 1079 results with 40 API calls
Retrieved 1087 results with 44 API calls
Retrieved 1087 results with 48 API calls
Retrieved 1087 results with 52 API calls
Retrieved 1165 results with 56 API calls
Retrieved 1235 results with 60 API calls
Retrieved 1319 results with 64 API calls
Retrieved 1448 results with 68 API calls
Retrieved 1648 results with 72 API calls
Retrieved 1848 results with 76 API calls
Retrieved 2048 results with 80 API calls
Retrieved 2178 results with 84 API calls
Retrieved 2250 results with 88 API calls
Retrieved 2259 results with 92 API calls
Retrieved 22

In [ ]:
# This code was for testing and looking at how we wanted to pull the data

# print(response['businesses'][0]['name'])
# print(f"{response['businesses'][0]['rating']} ({response['businesses'][0]['review_count']})")
# print(response['businesses'][0]['price'])
# print(response['businesses'][0]['display_phone'])
# print(f"Latitiude: {response['businesses'][0]['coordinates']['latitude']}, Longitude: {response['businesses'][0]['coordinates']['longitude']}")

# test_categories = ''
# for alias in response['businesses'][0]['categories']:
#     test_categories = test_categories + '/' + alias['title']
# print(test_categories)

# test_transactions = ''
# for type in response['businesses'][0]['transactions']:
#     test_transactions = test_transactions + '/' + type
# print(test_transactions)

In [4]:
Restaurant_id = 16995 # edit this to match id at end of csv
restaurant_data = [] # List to hold restaurant dictionaries
for restaurant in results:
    name = restaurant['name'] # get restaurant name
    rating = restaurant['rating'] # get restaurant rating
    review_count = restaurant['review_count'] # get restaurant review count
    try:
        price = restaurant['price']  # get restaurant price, if it fails ...
    except KeyError:
        price = '???' # set price to ???
    phone_number = restaurant['display_phone'] # get restaurant phone number
    latitiude = restaurant['coordinates']['latitude'] # get restaurant lat
    longitude = restaurant['coordinates']['longitude'] # get restaurant lon
    Restaurant_id += 1

    # merge all categories into one line seperated by /
    categories = ''
    x = 0
    for alias in restaurant['categories']:
        if x == 0:
            categories = alias['title']
            x += 1
        else:
            categories = categories + '/' + alias['title']

    # merge all transaction types into one line seperated by /
    transactions = ''
    x = 0
    for type in restaurant['transactions']:
        if x == 0:
            transactions = type
            x += 1
        else:
            transactions = transactions + '/' + type

    # Build a dictionary for the restaurant and add it to the list
    restaurant_data.append({'Restaurant_id':Restaurant_id,
                            'Name':name,
                            'Rating':rating,
                            'Review_Count':review_count,
                            'Price':price,
                            'Phone_Number':phone_number,
                            'Latitude':latitiude,
                            'Longitude':longitude,
                            'Categories':categories,
                            'Transactions':transactions})

# Create a DF from list
restaurant_data_df = pd.DataFrame(restaurant_data)
restaurant_data_df = restaurant_data_df.set_index('Restaurant_id')
restaurant_data_df

,Name,Rating,Review_Count,Price,Phone_Number,Latitude,Longitude,Categories,Transactions
Restaurant_id,,,,,,,,,
16996,Marriott,0.0,0,???,,40.831123,-74.014473,Restaurants,
16997,Tango Steak,4.3,386,$$,(201) 870-6161,40.842350,-74.044420,Argentine/Steakhouses/Cocktail Bars,delivery
16998,Viking Restaurant,0.0,0,???,,40.845010,-74.041730,Restaurants,pickup
16999,Subway,1.3,4,???,(201) 440-0015,40.844787,-74.041598,Sandwiches/Fast Food,delivery/pickup
17000,Nikki's Bar & Grill,4.2,64,$$,(201) 518-2883,40.845343,-74.041610,Sports Bars/Spanish/Cocktail Bars,delivery/pickup
...,...,...,...,...,...,...,...,...,...
20771,Gk Sweet Foods,0.0,0,???,(914) 788-3697,40.896474,-73.780374,Desserts/Pop-Up Restaurants,
20772,Tokyo Bay,0.0,0,???,(914) 576-9838,40.898730,-73.777790,Chinese,pickup
20773,Matrix on the go,1.0,1,???,(914) 236-2817,40.897668,-73.779384,Food Trucks/Caribbean/Caterers,


In [5]:
# Check for duplicates and display them
duplicates = restaurant_data_df.duplicated(keep=False)
restaurant_data_df[duplicates].sort_values('Name')

,Name,Rating,Review_Count,Price,Phone_Number,Latitude,Longitude,Categories,Transactions
Restaurant_id,,,,,,,,,
17855,Domino's Pizza,1.6,51,$,(718) 822-6010,40.845325,-73.864639,Pizza/Chicken Wings/Sandwiches,
19144,Domino's Pizza,1.6,51,$,(718) 822-6010,40.845325,-73.864639,Pizza/Chicken Wings/Sandwiches,
18148,Goldmans Treasures Giftware Plus,0.0,0,???,(201) 807-9051,40.863400,-74.031790,Restaurants,
19334,Goldmans Treasures Giftware Plus,0.0,0,???,(201) 807-9051,40.863400,-74.031790,Restaurants,
17862,Joy Garden Chinese Restaurant,2.4,5,$$$$,(718) 518-1242,40.845310,-73.866470,Chinese,pickup/delivery
19148,Joy Garden Chinese Restaurant,2.4,5,$$$$,(718) 518-1242,40.845310,-73.866470,Chinese,pickup/delivery
17863,Tastes Carribbean Restaurant,0.0,0,???,(718) 824-0022,40.845260,-73.866720,Restaurants,pickup
19150,Tastes Carribbean Restaurant,0.0,0,???,(718) 824-0022,40.845260,-73.866720,Restaurants,pickup


In [6]:
# Remove second copy of duplicates
restaurant_data_clean = restaurant_data_df.drop_duplicates()
restaurant_data_clean

,Name,Rating,Review_Count,Price,Phone_Number,Latitude,Longitude,Categories,Transactions
Restaurant_id,,,,,,,,,
16996,Marriott,0.0,0,???,,40.831123,-74.014473,Restaurants,
16997,Tango Steak,4.3,386,$$,(201) 870-6161,40.842350,-74.044420,Argentine/Steakhouses/Cocktail Bars,delivery
16998,Viking Restaurant,0.0,0,???,,40.845010,-74.041730,Restaurants,pickup
16999,Subway,1.3,4,???,(201) 440-0015,40.844787,-74.041598,Sandwiches/Fast Food,delivery/pickup
17000,Nikki's Bar & Grill,4.2,64,$$,(201) 518-2883,40.845343,-74.041610,Sports Bars/Spanish/Cocktail Bars,delivery/pickup
...,...,...,...,...,...,...,...,...,...
20771,Gk Sweet Foods,0.0,0,???,(914) 788-3697,40.896474,-73.780374,Desserts/Pop-Up Restaurants,
20772,Tokyo Bay,0.0,0,???,(914) 576-9838,40.898730,-73.777790,Chinese,pickup
20773,Matrix on the go,1.0,1,???,(914) 236-2817,40.897668,-73.779384,Food Trucks/Caribbean/Caterers,


In [7]:
# Append dataframe data to a csv file
restaurant_data_clean.to_csv("data/restaurant_data.csv",mode='a',header=False,index='Restaurant_id')

In [ ]:
# Read in data from the new york city health csv file

# health_df = pd.read_csv("data/DOHMH_New_York_City_Restaurant_Inspection_Results_20241120.csv")
# health_df.head()

In [ ]:
# Make sure phones numbers match bewteen both dataframes, rename columns and get rid of columns not needed

# health_df['PHONE'] = health_df['PHONE'].astype(str).str.replace(r'(\d{3})(\d{3})(\d{4})', r'(\1) \2-\3', regex=True)
# health_df[['Phone Number','Health Grade']] = health_df[['PHONE','GRADE']]
# columns_to_keep = ['Phone Number', 'Health Grade']
# health_df_cleaned = health_df.loc[:, columns_to_keep]
# health_df_cleaned

In [ ]:
# Get health grade for phone numbers from restaurant_data_clean found in health_df_cleaned

# restaurant_data_clean.loc[:,'Health Grade'] = restaurant_data_clean['Phone Number'].map(health_df_cleaned['Health Grade'])
# restaurant_data_clean

In [8]:
# read back in csv to make sure no duplicates and save to json
restaurant_data_csv = pd.read_csv("data/restaurant_data.csv", index_col='Restaurant_id')
restaurant_data_csv

,Name,Rating,Review_Count,Price,Phone_Number,Latitude,Longitude,Categories,Transactions
Restaurant_id,,,,,,,,,
0,Nino's Pizza,3.7,234,$,(718) 680-0222,40.619812,-74.032867,Pizza,pickup/delivery
1,Al Horno Lean Mexican Kitchen,4.3,51,$$,(347) 578-7492,40.619730,-74.032760,Mexican,pickup/delivery
2,Foodtown,2.9,130,$$,(718) 680-2884,40.619980,-74.032320,Grocery/Delis,NaN
3,Paneantico Bakery,3.5,475,$$,(718) 680-2347,40.619499,-74.033012,Bakeries/Sandwiches,pickup/delivery
4,Bridgeview Diner,2.5,343,$$,(718) 680-9818,40.620640,-74.031900,Diners/American/Breakfast & Brunch,pickup/delivery
...,...,...,...,...,...,...,...,...,...
20771,Gk Sweet Foods,0.0,0,???,(914) 788-3697,40.896474,-73.780374,Desserts/Pop-Up Restaurants,NaN
20772,Tokyo Bay,0.0,0,???,(914) 576-9838,40.898730,-73.777790,Chinese,pickup
20773,Matrix on the go,1.0,1,???,(914) 236-2817,40.897668,-73.779384,Food Trucks/Caribbean/Caterers,NaN


In [ ]:
# fixing column name issue
# restaurant_data_csv = restaurant_data_csv.rename(columns={'Review Count':'Review_Count','Phone Number':'Phone_Number', 'Latitiude':'Latitude'})
# restaurant_data_csv

In [ ]:
# Fixing id issue
# restaurant_data_csv = restaurant_data_csv.rename(columns={'Unnamed: 0':'Restaurant_id'})
# restaurant_data_csv

In [ ]:
# Fixing id issue
# restaurant_data_csv['Restaurant_id'] = range(len(restaurant_data_csv))
# restaurant_data_csv = restaurant_data_csv.set_index('Restaurant_id')
# restaurant_data_csv

In [ ]:
# Fixing id issue
# restaurant_data_csv.to_csv("data/restaurant_data.csv",index='Restaurant_id')

In [9]:
# check for duplicates after csv update
duplicates = restaurant_data_csv.duplicated(keep=False)
restaurant_data_csv[duplicates].sort_values('Name')

,Name,Rating,Review_Count,Price,Phone_Number,Latitude,Longitude,Categories,Transactions
Restaurant_id,,,,,,,,,
4332,1031 Fulton,0.0,0,???,(718) 277-7581,40.683286,-73.872865,Pizza,NaN
5778,1031 Fulton,0.0,0,???,(718) 277-7581,40.683286,-73.872865,Pizza,NaN
16852,Acogedor Cibao,0.0,0,???,(718) 328-6956,40.827270,-73.886529,Restaurants,NaN
17633,Acogedor Cibao,0.0,0,???,(718) 328-6956,40.827270,-73.886529,Restaurants,NaN
4437,Big Well Bakery & Halal Resturant,2.5,26,???,(718) 480-3875,40.683190,-73.834150,Halal/Bakeries/Tapas/Small Plates,delivery/pickup
5934,Big Well Bakery & Halal Resturant,2.5,26,???,(718) 480-3875,40.683190,-73.834150,Halal/Bakeries/Tapas/Small Plates,delivery/pickup
11754,IShanghai,3.9,8,???,(718) 316-6410,40.755302,-73.768300,Shanghainese,NaN
13793,IShanghai,3.9,8,???,(718) 316-6410,40.755302,-73.768300,Shanghainese,NaN
16850,King Sunshine Jerk Center 2,2.6,13,???,(347) 862-4020,40.827240,-73.886580,Juice Bars & Smoothies/American,pickup/delivery


In [13]:
# remove duplicates
restaurant_data_csv_clean = restaurant_data_csv.drop_duplicates()
restaurant_data_csv_clean

,Name,Rating,Review_Count,Price,Phone_Number,Latitude,Longitude,Categories,Transactions
Restaurant_id,,,,,,,,,
0,Nino's Pizza,3.7,234,$,(718) 680-0222,40.619812,-74.032867,Pizza,pickup/delivery
1,Al Horno Lean Mexican Kitchen,4.3,51,$$,(347) 578-7492,40.619730,-74.032760,Mexican,pickup/delivery
2,Foodtown,2.9,130,$$,(718) 680-2884,40.619980,-74.032320,Grocery/Delis,NaN
3,Paneantico Bakery,3.5,475,$$,(718) 680-2347,40.619499,-74.033012,Bakeries/Sandwiches,pickup/delivery
4,Bridgeview Diner,2.5,343,$$,(718) 680-9818,40.620640,-74.031900,Diners/American/Breakfast & Brunch,pickup/delivery
...,...,...,...,...,...,...,...,...,...
20771,Gk Sweet Foods,0.0,0,???,(914) 788-3697,40.896474,-73.780374,Desserts/Pop-Up Restaurants,NaN
20772,Tokyo Bay,0.0,0,???,(914) 576-9838,40.898730,-73.777790,Chinese,pickup
20773,Matrix on the go,1.0,1,???,(914) 236-2817,40.897668,-73.779384,Food Trucks/Caribbean/Caterers,NaN


In [14]:
# re-write to csv
restaurant_data_csv_clean.to_csv("data/restaurant_data.csv",index='Restaurant_id')

In [15]:
# Save data as .json
restaurant_data_json = restaurant_data_csv_clean.reset_index()
restaurant_data_json.to_json('data/restaurant_data.json', orient='records', lines=False)

In [16]:
# Creating a .js file from our data for map usage
json_file_path = 'data/restaurant_data.json'
with open(json_file_path, 'r') as json_file:
    json_data = json.load(json_file)

variable_name = 'restaurant_data'
js_content = f'export const {variable_name} = {json.dumps(json_data, indent=4)};'

js_file_path = 'data/restaurant_data.js'
with open(js_file_path, 'w') as js_file:
    js_file.write(js_content)

In [2]:
# Test json file to make sure it has the right number of lines
test_json_df = pd.read_json('data/restaurant_data.json')
test_json_df

,Restaurant_id,Name,Rating,Review_Count,Price,Phone_Number,Latitude,Longitude,Categories,Transactions
0,0,Nino's Pizza,3.7,234,$,(718) 680-0222,40.619812,-74.032867,Pizza,pickup/delivery
1,1,Al Horno Lean Mexican Kitchen,4.3,51,$$,(347) 578-7492,40.619730,-74.032760,Mexican,pickup/delivery
2,2,Foodtown,2.9,130,$$,(718) 680-2884,40.619980,-74.032320,Grocery/Delis,None
3,3,Paneantico Bakery,3.5,475,$$,(718) 680-2347,40.619499,-74.033012,Bakeries/Sandwiches,pickup/delivery
4,4,Bridgeview Diner,2.5,343,$$,(718) 680-9818,40.620640,-74.031900,Diners/American/Breakfast & Brunch,pickup/delivery
...,...,...,...,...,...,...,...,...,...,...
20755,20771,Gk Sweet Foods,0.0,0,???,(914) 788-3697,40.896474,-73.780374,Desserts/Pop-Up Restaurants,None
20756,20772,Tokyo Bay,0.0,0,???,(914) 576-9838,40.898730,-73.777790,Chinese,pickup
20757,20773,Matrix on the go,1.0,1,???,(914) 236-2817,40.897668,-73.779384,Food Trucks/Caribbean/Caterers,None
20758,20774,Sandwich King,3.9,9,???,(914) 600-8801,40.898720,-73.777966,Burgers/Sandwiches/Steakhouses,pickup/delivery
